In [1]:
import torch
from torch.nn.functional import pad

In [2]:
dim = 2
# mock embedding matrix 1
seq_len1 = 6
matrix1 = torch.randn(seq_len1, dim)
# mock embedding matrix 2
seq_len2 = 4
matrix2 = torch.randn(seq_len2, dim)
# ensure matrices only contain non-negative values
matrix1 = torch.abs(matrix1)
matrix2 = torch.abs(matrix2)

In [3]:
targets1 = torch.tensor([-1,-1,4,-1,-1,2])
targets2 = torch.tensor([-1,1,-1,8])

In [4]:
targets = [targets1, targets2]

In [5]:
for i, target in enumerate(targets):
    targets[i] = torch.roll(target,-1)
    targets[i][-1] = -1

In [6]:
targets = torch.nested.nested_tensor(targets)

/run/media/john/Secondary/Documents/Cornell/Spring 2024/ORIE 4741 Learning From Data/Project/proj/lib64/python3.11/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(


In [7]:
targets = torch.nested.to_padded_tensor(targets, -1)

In [30]:
targets

tensor([[-1,  4, -1, -1,  2, -1],
        [ 1, -1,  8, -1, -1, -1]])

In [8]:
matrices = [matrix1, matrix2]

In [9]:
longest_seq_len = max([matrix.size(0) for matrix in matrices])

In [10]:
seq_lens = [matrix.size(0) for matrix in matrices]

In [11]:
for i in range(len(matrices)):
    m = matrices[i]
    matrices[i] = pad(m, (0, 0, 0, longest_seq_len - m.size(0)), value=-100000)

In [12]:
matrices[-1][-1] = 1

In [13]:
from torch.nn.functional import scaled_dot_product_attention as sdpa

In [14]:
attn_matrices = []

In [15]:
naive_attn = []

In [16]:
for i in range(len(seq_lens)):
    causal_mask = torch.tril(torch.ones(longest_seq_len, longest_seq_len), diagonal=0)
    causal_mask[:, seq_lens[i]: ] = 0
    attn_matrices.append(causal_mask.bool())

In [17]:
for i in range(len(seq_lens)):
    causal_mask = torch.tril(torch.ones(longest_seq_len, longest_seq_len), diagonal=0)
    naive_attn.append(causal_mask.bool())

In [18]:
attn_matrices = torch.stack(attn_matrices)
naive_attn = torch.stack(naive_attn)

In [19]:
matrices = torch.stack(matrices)

In [20]:
q = torch.rand_like(matrices)
k = torch.rand_like(matrices)

In [21]:
result = sdpa(q, k, matrices, naive_attn)

In [22]:
matrices

tensor([[[ 1.6257e+00,  6.9774e-01],
         [ 1.2349e+00,  2.2260e-01],
         [ 2.0616e-01,  4.4840e-01],
         [ 4.5609e-01,  4.9041e-01],
         [ 1.8428e+00,  4.6549e-01],
         [ 1.4216e+00,  9.1426e-01]],

        [[ 4.8778e-01,  6.6217e-01],
         [ 1.1724e+00,  1.5859e+00],
         [ 7.9270e-02,  6.4565e-01],
         [ 3.5792e-01,  1.0977e+00],
         [-1.0000e+05, -1.0000e+05],
         [ 1.0000e+00,  1.0000e+00]]])

In [23]:
result

tensor([[[ 1.6257e+00,  6.9774e-01],
         [ 1.4354e+00,  4.6639e-01],
         [ 9.7711e-01,  4.4738e-01],
         [ 7.9450e-01,  4.6234e-01],
         [ 1.0369e+00,  4.6194e-01],
         [ 1.1020e+00,  5.3125e-01]],

        [[ 4.8778e-01,  6.6217e-01],
         [ 8.3678e-01,  1.1331e+00],
         [ 5.8335e-01,  9.6802e-01],
         [ 5.0429e-01,  9.8661e-01],
         [-2.2155e+04, -2.2154e+04],
         [-1.8697e+04, -1.8697e+04]]])

In [24]:
sdpa(q, k, matrices, attn_matrices)

tensor([[[1.6257, 0.6977],
         [1.4354, 0.4664],
         [0.9771, 0.4474],
         [0.7945, 0.4623],
         [1.0369, 0.4619],
         [1.1020, 0.5312]],

        [[0.4878, 0.6622],
         [0.8368, 1.1331],
         [0.5833, 0.9680],
         [0.5043, 0.9866],
         [0.5342, 1.0096],
         [0.5392, 1.0137]]])

In [25]:
before_roll = torch.tensor([0,-1,-1,3,-1,-1,-1,1])

In [26]:
rolled = torch.roll(before_roll,-1)

In [27]:
rolled

tensor([-1, -1,  3, -1, -1, -1,  1,  0])

In [28]:
rolled[-1] = -1

In [29]:
rolled

tensor([-1, -1,  3, -1, -1, -1,  1, -1])